In [1]:
import pandas as pd
import sqlite3
db_file = 'sqlite_db_pythonsqlite.db'
conn = sqlite3.connect(db_file)

**Question 1**: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [2]:
query1 = '''SELECT name AS 'Facility Name' 
FROM Facilities WHERE membercost <> 0;'''
Q1 = pd.read_sql_query(query1, conn)
Q1

,Facility Name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


**Question 2**: How many facilities do not charge a fee to members?

In [3]:
query2 = '''SELECT 
COUNT(name) AS 'Facilities without members fees' 
FROM Facilities WHERE membercost = 0;'''
Q2 = pd.read_sql_query(query2, conn, index_col = None)
Q2

,Facilities without members fees
0,4


**Question 3**: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [4]:
query3 = '''SELECT 
facid, 
name AS 'Facility Name', 
membercost AS 'Member Cost', 
monthlymaintenance AS 'Monthly Maintenance' 
FROM Facilities 
WHERE (membercost <>0) AND (membercost < ( monthlymaintenance * 0.2 ));'''
Q3 = pd.read_sql_query(query3, conn, index_col = None)
Q3

,facid,Facility Name,Member Cost,Monthly Maintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


**Question 4**: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [5]:
query4 = '''SELECT * FROM `Facilities` 
WHERE facid IN ( 1, 5 );'''
Q4 = pd.read_sql_query(query4, conn, index_col = None)
Q4

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


**Question 5**: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [6]:
query5 = '''SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance <=100 THEN 'cheap'
     ELSE 'expensive' END AS 'cheap/expensive'
FROM Facilities;'''
Q5 = pd.read_sql_query(query5, conn, index_col = None)
Q5

,name,monthlymaintenance,cheap/expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


**Question 6**: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [7]:
query6 = '''SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);''' 
Q6 = pd.read_sql_query(query6, conn, index_col = None)
Q6

,firstname,surname
0,Darren,Smith


**Question 7**: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [8]:
query7 = '''SELECT DISTINCT(CONCAT_WS(" ", M.firstname, M.surname)) AS fullname, F.name
FROM Bookings AS B
INNER JOIN Members AS M
ON B.memid = M.memid
INNER JOIN Facilities AS F
ON B.facid = F.facid
WHERE name LIKE '%Tennis Court%'
ORDER BY fullname;'''
Q7= pd.read_sql_query(query7, conn, index_col = None)
Q7

,fullname,name
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 1
8,David Farrell,Tennis Court 2
9,David Jones,Tennis Court 2


**Question 8**: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [9]:
query8 = '''SELECT 
name AS facilityname,
CASE WHEN memid = 0 THEN 'Guest'
	 ELSE CONCAT_WS(" ", firstname, surname) END AS fullname,
CASE WHEN memid = 0 THEN guestcost * slots
ELSE membercost * slots END AS totalcost
FROM Bookings
INNER JOIN Facilities
USING (facid)
INNER JOIN Members
USING (memid)
WHERE starttime LIKE '%2012-09-14%' AND (CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END > 30)
ORDER BY totalcost DESC;'''
Q8= pd.read_sql_query(query8, conn)
Q8

,facilityname,fullname,totalcost
0,Massage Room 2,Guest,320.0
1,Massage Room 1,Guest,160.0
2,Massage Room 1,Guest,160.0
3,Massage Room 1,Guest,160.0
4,Tennis Court 2,Guest,150.0
5,Tennis Court 1,Guest,75.0
6,Tennis Court 1,Guest,75.0
7,Tennis Court 2,Guest,75.0
8,Squash Court,Guest,70.0
9,Massage Room 1,Jemima Farrell,39.6


**Question 9**: This time, produce the same result as in Q8, but using a subquery.

In [10]:
query9 = '''SELECT facilityname, CASE WHEN j.memid = 0 THEN 'Guest' ELSE CONCAT_WS(" ", firstname, surname) END AS fullname, totalcost
FROM Members AS m
INNER JOIN
(SELECT memid, name AS facilityname,
(CASE WHEN memid = 0 THEN (guestcost * slots) ELSE (membercost * slots) END) AS totalcost
FROM Bookings
INNER JOIN Facilities
USING (facid)
WHERE starttime LIKE '%2012-09-14%' AND (CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) > 30) AS j
ON m.memid = j.memid
ORDER BY totalcost DESC;'''
Q9 = pd.read_sql_query(query9, conn)
Q9

,facilityname,fullname,totalcost
0,Massage Room 2,Guest,320.0
1,Massage Room 1,Guest,160.0
2,Massage Room 1,Guest,160.0
3,Massage Room 1,Guest,160.0
4,Tennis Court 2,Guest,150.0
5,Tennis Court 1,Guest,75.0
6,Tennis Court 1,Guest,75.0
7,Tennis Court 2,Guest,75.0
8,Squash Court,Guest,70.0
9,Massage Room 1,Jemima Farrell,39.6


**Question 10**: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [11]:
query10 = '''SELECT
name,
SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue
FROM Bookings
INNER JOIN Facilities
USING (facid)
GROUP BY facid
HAVING revenue<1000
ORDER BY revenue;'''

Q10 = pd.read_sql_query(query10, conn)
Q10

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


**Question 11**: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [13]:
query11 = '''SELECT m1.surname AS 'Member Surname', 
m1.firstname AS 'Member First Name', 
CONCAT_WS( " ", m2.surname, m2.firstname ) AS 'Recommended By'
FROM Members AS m1
INNER JOIN Members AS m2 ON m1.recommendedby = m2.memid
ORDER BY `m1`.`surname` ASC;'''
Q11 = pd.read_sql_query(query11, conn)
Q11

,Member Surname,Member First Name,Recommended By
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren
5,Coplin,Joan,Baker Timothy
6,Crumpet,Erica,Smith Tracy
7,Dare,Nancy,Joplette Janice
8,Genting,Matthew,Butters Gerald
9,Hunt,John,Purview Millicent


**Quesiton 12**: Find the facilities with their usage by member, but not guests

In [19]:
query12 = '''SELECT name AS "Facility Name", SUM(slots) AS "Member Usage"
FROM Bookings
INNER JOIN Facilities
USING (facid)
WHERE memid <> 0
GROUP BY facid;
'''
Q12 = pd.read_sql_query(query12, conn)
Q12

,Facility Name,Member Usage
0,Tennis Court 1,957
1,Tennis Court 2,882
2,Badminton Court,1086
3,Table Tennis,794
4,Massage Room 1,884
5,Massage Room 2,54
6,Squash Court,418
7,Snooker Table,860
8,Pool Table,856


**Quesiton 13**: Find the facilities with their usage by member, but not guests

In [26]:
query13 = '''SELECT name AS 'Facility Name', strftime('%m', starttime) AS Month, SUM(slots) AS 'Monthly Usage'
FROM Bookings
INNER JOIN Facilities
USING (facid)
WHERE memid <> 0
GROUP BY facid, month;'''
Q13 = pd.read_sql_query(query13, conn)
Q13

,Facility Name,Month,Monthly Usage
0,Tennis Court 1,07,201
1,Tennis Court 1,08,339
2,Tennis Court 1,09,417
3,Tennis Court 2,07,123
4,Tennis Court 2,08,345
5,Tennis Court 2,09,414
6,Badminton Court,07,165
7,Badminton Court,08,414
8,Badminton Court,09,507
9,Table Tennis,07,98
